
Hourly Data for 2014 Jan - 2023 Dec 
- ["PG", "UL", "JNJ", "PFE", "XOM", "CVX", "JPM", "GS"]
- Apple (AAPL)
- Microsoft (MSFT)
- Procter & Gamble (PG)
- Unilever (UL)
- Johnson & Johnson (JNJ)
- Pfizer (PFE)
- ExxonMobil (XOM)
- Chevron (CVX)
- JPMorgan Chase (JPM)
- Goldman Sachs (GS)

Before running
- create an empty "log.csv" with column "symbol_year_month"
- create an empty "stocks.csv" with columns ['date', 'stock_name', 'open', 'high', 'low', 'close', 'volume']

In [ ]:
# imports
import requests
import pandas as pd
import os

# config
API_KEY = "" #replace with api key from https://www.alphavantage.co/
STOCKS_CSV = "stocks.csv"
LOGGING_CSV = "log.csv"

# create lists of companies and dates to search up
symbol_list = ["AAPL", "PG", "UL", "JNJ", "PFE"]

start_date = '2014-01-01'
end_date = '2023-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='M')
year_month_list = date_range.strftime('%Y-%m').tolist()

In [ ]:
# test api call
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=60min&month=2020-07&outputsize=full&apikey={API_KEY}"
response = requests.get(url)
print(response.ok, response, response.json())

In [41]:
def fetch_data_if_not_exist(symbol_list, year_month_list, api_key = API_KEY, stocks_csv = STOCKS_CSV, log_csv=LOGGING_CSV):
    """
    maintains a log.csv of the successful api requests, 
    when looping through symbol (company) list and year_month_list, if symbol(company)_year_month is not in the log, make api request 
    add the data back into stocks.csv
    """

    log_list = pd.read_csv(log_csv).symbol_year_month.to_list()

    df_stocks = pd.read_csv(stocks_csv)

    api_limit = 0

    for symbol in symbol_list:
    
        for year_month in year_month_list:

            # check if the year-month for the symbol has already been done 
            if f"{symbol}_{year_month}" in log_list:
                continue
            
            if api_limit < 25:
                print(f"checking {symbol}_{year_month}")

                # make api request
                url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=60min&month={year_month}&outputsize=full&apikey={api_key}"
                response = requests.get(url)

                # parse json response into dict
                data = response.json()

                # convert to df
                df = pd.DataFrame.from_dict(data["Time Series (60min)"], orient='index')

                # rename columns + set data type of numeric columns
                numeric_columns = ['open', 'high', 'low', 'close', 'volume']
                df.columns = numeric_columns
                df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

                # date is currently the index column, reset index and rename date column
                df.index.name = 'date'
                df.reset_index(inplace=True)
                
                # fill up stock_name column
                df.insert(1, 'stock_name', symbol)

                # add to main dataset
                df_stocks = pd.concat([df_stocks, df], axis = 0, ignore_index=True)
                log_list += [f"{symbol}_{year_month}"]

                api_limit += 1

    # save as csv
    df_stocks.to_csv(stocks_csv, index=False)
    df_log = pd.DataFrame(log_list, columns=['symbol_year_month'])
    df_log.to_csv(log_csv, index = False)

    return df_stocks.shape

In [74]:
# just keep runnning this, if the output is df.shape only, it should mean that all data has been retrieved
# if the output fails just change the server/ disconnect and reconnect

fetch_data_if_not_exist(symbol_list, year_month_list)

(162719, 7)

In [2]:
df_stocks = pd.read_csv("stocks.csv")
df_stocks.to_csv("AAPL_PG_UL_JNJ_PFE_2014_2023.csv", index=False)

In [3]:
# separate data based on company and save as csv
full_df = pd.read_csv("AAPL_PG_UL_JNJ_PFE_2014_2023.csv")

for symbol in symbol_list:
    symbol_df = full_df[full_df['stock_name'] == symbol]
    symbol_df.to_csv(f"./company_hourly/{symbol}.csv", index=False)
